In [1]:
import re
from jamo import h2j, j2h, j2hcj
from konlpy.tag import Kkma, Komoran

kkma = Kkma()
komoran = Komoran()
tokenizer = komoran

In [2]:
irregular_conjugation_stems = ['걷', '긷', '깨닫', '눋', '닫', '듣', '묻', '붇', '싣', '일컫', '가깝', '가볍', '간지럽', '굽', '그립', '깁', '껄끄럽', '노엽', '더럽', '덥', '맵', '메스껍', '무겁', '반갑', '부끄럽', '사납', '서럽', '쑥스럽', '줍', '긋', '낫', '붓', '잇', '잣', '젓', '짓', ]

In [3]:
def verb_exception_handler(verb):
    if verb == '나서':
        return '나'
    else:
        return verb

In [4]:
def verb_to_noun(verb):
    
    """
    한국어의 용언을 명사형으로 변환하는 함수입니다.
    용언(동사, 형용사)의 어근을 받아 명사형으로 반환합니다.
        
    """
    
    verb = verb_exception_handler(verb)

    eum = '음'  # 명사형 전성 어미 '음'
    m = h2j('음')[-1]  # 명사형 전성 어미 'ㅁ'

    d = h2j('ㄷ')  # ㄷ 불규칙 활용
    s = h2j('ㅅ')  # ㅅ 불규칙 활용
    b = h2j('ㅂ')  # ㅂ 불규칙 활용

    last_syllables = h2j(verb[-1])  # 용언의 마지막 자모군
    final = j2hcj(last_syllables[-1])  # 마지막 자모군의 끝글자

    pattern = re.compile(r'[ㅏ-ㅣ]')
    has_final = not bool(pattern.match(final))  # 마지막 자모군에 종성 존재 여부

    if has_final and final != b:  # 종성이 존재하는 경우 / ㅂ 불규칙 활용의 경우 종성이 없는 경우로 취급
        if final == d and verb in irregular_conjugation_stems:  # ㄷ 불규칙 활용 적용
            modified = last_syllables[:-1] + h2j('ㄹ')
            return verb[:-1] + j2h(*modified) + eum
        elif final == s and verb in irregular_conjugation_stems:  # ㅅ 불규칙 활용 적용
            modified = last_syllables[:-1]
            return verb[:-1] + j2h(*modified) + eum
        else:  # 종성이 존재하나 불규칙 활용이 아닌 경우
            modified = last_syllables
            return verb[:-1] + j2h(*modified) + eum
    else:  # 종성이 존재하지 않는 경우
        if final == b and verb in irregular_conjugation_stems:  # ㅂ 불규칙 활용
            modified = h2j('우') + m
            return verb[:-1] + j2h(*last_syllables[:-1]) + j2h(*modified)
        else:  # 종성이 존재하지 않으며 불규칙 활용이 아닌 경우
            modified = last_syllables + m
            return verb[:-1] + j2h(*modified)

# 보다 / 돌다 / 걷다 / 긋다 / 가깝다 / 반갑다 / 넘어지

print(verb_to_noun('보'))
print(verb_to_noun('돌'))
print(verb_to_noun('걷'))
print(verb_to_noun('긋'))
print(verb_to_noun('가깝'))
print(verb_to_noun('반갑'))
print(verb_to_noun('넘어지'))

봄
돌음
걸음
그음
가까움
반가움
넘어짐


In [5]:
def extract_keywords(sentence):
    
    """
    문장을 입력으로 받아서 형태소 분석을 수행하고, 명사(Noun)와 동사(Verb)를 추출하는 함수
    동사 외에도 연결어미, 동사파생접미사, 몇몇 조사를 추출합니다.
    이후 마지막 토큰이 명사형 혹은 동사의 어근이 아닌 경우 삭제합니다.
    tokenizer별 형태소 분류표: https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0

    """
    pos = tokenizer.pos(sentence)
    # keywords_w_pos = [[word, tag] for word, tag in pos if tag in ['NNG', 'NNP', 'VV', 'VA', 'VX']]
    keywords_w_pos = [[word, tag] for word, tag in pos if tag in ['NNG', 'NNP', 'VV', 'VA', 'VX', 'EC', 'XSV', 'JKO', 'JKB', 'JKS']]
    
    while keywords_w_pos[-1][1] not in ['VV', 'VA', 'VX', 'NNG', 'NNP']:
        keywords_w_pos.pop(-1)

    print(keywords_w_pos)  # for debugging

    return keywords_w_pos

def sentence_to_noun_verb(sentence):

    """
    "사다리가 쓰러져서" 문장을 "사다리가 쓰러짐" 문자열로 변환하는 함수
    extract_keywords를 통해 얻은 keywords_w_pos에 대한 표현 정규화 작업을 수행합니다.
    리스트의 마지막 요소가 동사의 어근인 경우만 수행합니다. (['VV', 'VA', 'VX'])

    """
    keywords_w_pos = extract_keywords(sentence)

    # for item in keywords_w_pos:
    #     if item[1] in ['VV', 'VA', 'VX']:
    #         item[0] = verb_to_noun(item[0])
    
    if keywords_w_pos[-1][1] in ['VV', 'VA', 'VX']:
        keywords_w_pos[-1][0] = verb_to_noun(keywords_w_pos[-1][0])
    result = [word for word, tag in keywords_w_pos]
    head = ''.join(result[:-1])
    tail = result[-1]
    return ' '.join([head, tail])
    # return ' '.join(result)

In [9]:
test_cases = ['발판이 쓰러져서', '사다리가 넘어지다', '사다리가 넘어져서', '사다리가 넘어져', '발판이 쓰러져', '화재 발생', '배탈이 나서', '배탈이 났다']

# for item in test_cases:
#     result = extract_keywords(item)
#     print(result)

for item in test_cases:
    result = sentence_to_noun_verb(item)
    print(item, '>>>', result)
    print()

[['발판', 'NNG'], ['이', 'JKS'], ['쓰러지', 'VV']]
발판이 쓰러져서 >>> 발판이 쓰러짐

[['사다리', 'NNP'], ['가', 'JKS'], ['넘어지', 'VV']]
사다리가 넘어지다 >>> 사다리가 넘어짐

[['사다리', 'NNP'], ['가', 'JKS'], ['넘어지', 'VV']]
사다리가 넘어져서 >>> 사다리가 넘어짐

[['사다리', 'NNP'], ['가', 'JKS'], ['넘어지', 'VV']]
사다리가 넘어져 >>> 사다리가 넘어짐

[['발판', 'NNG'], ['이', 'JKS'], ['쓰러지', 'VV']]
발판이 쓰러져 >>> 발판이 쓰러짐

[['화재', 'NNP'], ['발생', 'NNG']]
화재 발생 >>> 화재 발생

[['배탈', 'NNG'], ['이', 'JKS'], ['나서', 'VV']]
배탈이 나서 >>> 배탈이 남

[['배탈', 'NNG'], ['이', 'JKS'], ['나', 'VV']]
배탈이 났다 >>> 배탈이 남



In [7]:
tokenizer.pos('결속하지')
tokenizer.pos('발판으로')

[('발판', 'NNG'), ('으로', 'JKB')]

In [8]:
test_cases = ['결속하지 아니하여', '결속하지 않아' , '착용하지 아니한 채로', '발생하지 않은', '보지 못하고', '보지 않고', '튀어 올라',  '들어가려다가', '결속하지 않아서', '키스토링에 걸리게', '발판이 넘어져서', '발판으로 넘어져서']

# for item in test_cases:
#     result = extract_keywords(item)
#     print(item, result)

for item in test_cases:
    result = sentence_to_noun_verb(item)
    print(item, '>>>', result)
    print()

[['결속', 'NNG'], ['하', 'XSV'], ['지', 'EC'], ['아니하', 'VX']]
결속하지 아니하여 >>> 결속하지 아니함

[['결속', 'NNG'], ['하', 'XSV'], ['지', 'EC'], ['않', 'VX']]
결속하지 않아 >>> 결속하지 않음

[['착용', 'NNG'], ['하', 'XSV'], ['지', 'EC'], ['아니하', 'VX']]
착용하지 아니한 채로 >>> 착용하지 아니함

[['발생', 'NNG'], ['하', 'XSV'], ['지', 'EC'], ['않', 'VX']]
발생하지 않은 >>> 발생하지 않음

[['보', 'VV'], ['지', 'EC'], ['못하', 'VX']]
보지 못하고 >>> 보지 못함

[['보', 'VV'], ['지', 'EC'], ['않', 'VX']]
보지 않고 >>> 보지 않음

[['튀', 'VV'], ['어', 'EC'], ['오르', 'VV']]
튀어 올라 >>> 튀어 오름

[['들어가', 'VV']]
들어가려다가 >>>  들어감

[['결속', 'NNG'], ['하', 'XSV'], ['지', 'EC'], ['않', 'VX']]
결속하지 않아서 >>> 결속하지 않음

[['키스', 'NNP'], ['토', 'NNG'], ['링', 'NNG'], ['에', 'JKB'], ['걸리', 'VV']]
키스토링에 걸리게 >>> 키스토링에 걸림

[['발판', 'NNG'], ['이', 'JKS'], ['넘어지', 'VV']]
발판이 넘어져서 >>> 발판이 넘어짐

[['발판', 'NNG'], ['으로', 'JKB'], ['넘어지', 'VV']]
발판으로 넘어져서 >>> 발판으로 넘어짐

